XOR CIPHER

In [65]:
from qiskit import QuantumCircuit,ClassicalRegister,QuantumRegister,transpile #,Aer,execute
from qiskit.providers.basic_provider import BasicSimulator
backend = BasicSimulator()

pt=QuantumRegister(4)
# ct=QuantumRegister(4)
k=QuantumRegister(4)
c=ClassicalRegister(4)
qc=QuantumCircuit(pt,k,c)
n=4

plain_text='0101'
key='1100'

# plain text
for i, char in enumerate(plain_text):
    if char == '1':
        qc.x(pt[i])


print('plain text')
qc.measure(pt,c)
tcirc = transpile(qc, backend)
result = backend.run(tcirc, shots=1024).result()
counts = result.get_counts()
print(counts)


# key
for i, char in enumerate(key):
    if char == '1':
        qc.x(k[i])


print('KEY')
qc.measure(k,c)
qc.measure(k,c)
tcirc = transpile(qc, backend)
result = backend.run(tcirc, shots=1024).result()
counts = result.get_counts()
print(counts)

# encryption algorithm
for i in range(4):
    qc.cx(k[i],pt[i])
# end of encryption

print('Cipher text')
qc.measure(pt,c)
tcirc = transpile(qc, backend)
result = backend.run(tcirc, shots=1024).result()
counts = result.get_counts()
print(counts)

plain text
{'1010': 1024}
KEY
{'0011': 1024}
Cipher text
{'1001': 1024}


GROVER SEARCH

In [66]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister#, BasicAer, execute, IBMQ
from qiskit.compiler import assemble
# from qiskit.tools.jupyter import *
from qiskit.visualization import *
# from ibm_quantum_widgets import CircuitComposer
# from qiskit.providers.aer import QasmSimulator, AerSimulator
import numpy as np
from numpy import pi

# Loading your IBM Quantum account(s)
# provider = IBMQ.load_account()

In [67]:
def load_input(qc, r_key, input_binary, n):
    for i in range(n):
        if (input_binary[i] == "1"):
            qc.x(r_key[i])

In [68]:
def load_key(qc, r_key, n):
    for i in range(n):
        qc.h(r_key[i])

In [69]:
def cipher(qc, r_key, r_text, n):
    for i in range(n):
        qc.cx(r_key[i], r_text[i])

In [70]:
def my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n):
    #Compute cipher text possibilities by XORing
    cipher(qc, r_key, r_text, n)
    
    #Checking whether the generated cipher text is equal to the given cipher text
    for i in range(n):
        qc.cx(r_cipher[i], r_ancilla[i])
        qc.cx(r_text[i], r_ancilla[i])
        qc.x(r_ancilla[i])
        
    #Set 'output' bit if the cipher text is matched
    qc.mcx(r_ancilla, r_output)
    
    qc.barrier()
    
    #Uncompute cipher to reset ancilla & plain text qubits
    #Reset ancilla qubit
    for i in range(n):
        qc.x(r_ancilla[i])
        qc.cx(r_cipher[i], r_ancilla[i])
        qc.cx(r_text[i], r_ancilla[i])
    qc.barrier()
    #Reset plain text by inverse cipher process
    cipher(qc, r_key, r_text, n)
    
    qc.barrier()
        

In [71]:
#The diffuser function is available in Qiskit
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    #Apply transformation |s> -> |00...0> (H gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    #Apply transformation |00...0> -> |11...1> (X gates)
    for qubit in range(nqubits):
        qc.x(qubit)    
    #Do multi-controlled Z gate
    qc.h(nqubits - 1)
    qc.mcx(list(range(nqubits - 1)), nqubits - 1) #multi-controlled Toffoli
    qc.h(nqubits - 1)
    #Apply transformation |11...1> -> |00...0> (X gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    #Apply transformation |00...0> -> |s> (H gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    #We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s
    

In [72]:
n = 4
r_text = QuantumRegister(n, 't')
r_key = QuantumRegister(n, 'k')
r_output = QuantumRegister(1, name = 'o')
r_cipher = QuantumRegister(n, 'c')
r_ancilla = QuantumRegister(n, 'a')

r_class = ClassicalRegister(n)

qc = QuantumCircuit(r_text, r_key, r_cipher, r_ancilla, r_output, r_class)
#loading plain text
plain_text = '0101'
load_input(qc, r_text, plain_text, n)

#Preparing key in a superposition state
load_key(qc, r_key, n)

#loading known cipher text
cipher_text = '1001'
load_input(qc, r_cipher, cipher_text, n)

#Preparing output qubit
qc.x(r_output)
qc.h(r_output)
qc.barrier()

#First Iteration
my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
qc.barrier()
qc.append(diffuser(n), r_key)
qc.barrier()

#Second Iteration
my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
qc.barrier()
qc.append(diffuser(n), r_key)
qc.barrier()

qc.measure(r_key, r_class)
qc.draw()


░ ┌───┐                                              ░      »
  t_0: ───────────░─┤ X ├─────────────────■────────────────────────────░──────»
       ┌───┐      ░ └─┬─┘┌───┐            │                            ░      »
  t_1: ┤ X ├──────░───┼──┤ X ├────────────┼────■───────────────────────░──────»
       └───┘      ░   │  └─┬─┘┌───┐       │    │                       ░      »
  t_2: ───────────░───┼────┼──┤ X ├───────┼────┼────■──────────────────░──────»
       ┌───┐      ░   │    │  └─┬─┘┌───┐  │    │    │                  ░      »
  t_3: ┤ X ├──────░───┼────┼────┼──┤ X ├──┼────┼────┼────■─────────────░──────»
       ├───┤      ░   │    │    │  └─┬─┘  │    │    │    │             ░      »
  k_0: ┤ H ├──────░───■────┼────┼────┼────┼────┼────┼────┼─────────────░──────»
       ├───┤      ░        │    │    │    │    │    │    │             ░      »
  k_1: ┤ H ├──────░────────■────┼────┼────┼────┼────┼────┼─────────────░──────»
       ├───┤      ░             │    │    │    │    │    │             ░      »
  k_2: ┤ H ├──────░─────────────■────┼────┼────┼────┼────┼─────────────░──────»
       ├───┤      ░                  │    │    │    │    │             ░      »
  k_3: ┤ H ├──────░──────────────────■────┼────┼────┼────┼─────────────░──────»
       ├───┤      ░                       │    │    │    │             ░      »
  c_0: ┤ X ├──────░───■───────────────────┼────┼────┼────┼─────────────░──────»
       └───┘      ░   │                   │    │    │    │             ░      »
  c_1: ───────────░───┼────■──────────────┼────┼────┼────┼─────────────░──────»
                  ░   │    │              │    │    │    │             ░      »
  c_2: ───────────░───┼────┼────■─────────┼────┼────┼────┼─────────────░──────»
       ┌───┐      ░   │    │    │         │    │    │    │             ░      »
  c_3: ┤ X ├──────░───┼────┼────┼────■────┼────┼────┼────┼─────────────░──────»
       └───┘      ░ ┌─┴─┐  │    │    │  ┌─┴─┐  │    │    │  ┌───┐      ░ ┌───┐»
  a_0: ───────────░─┤ X ├──┼────┼────┼──┤ X ├──┼────┼────┼──┤ X ├──■───░─┤ X ├»
                  ░ └───┘┌─┴─┐  │    │  └───┘┌─┴─┐  │    │  ├───┤  │   ░ ├───┤»
  a_1: ───────────░──────┤ X ├──┼────┼───────┤ X ├──┼────┼──┤ X ├──■───░─┤ X ├»
                  ░      └───┘┌─┴─┐  │       └───┘┌─┴─┐  │  ├───┤  │   ░ ├───┤»
  a_2: ───────────░───────────┤ X ├──┼────────────┤ X ├──┼──┤ X ├──■───░─┤ X ├»
                  ░           └───┘┌─┴─┐          └───┘┌─┴─┐├───┤  │   ░ ├───┤»
  a_3: ───────────░────────────────┤ X ├───────────────┤ X ├┤ X ├──■───░─┤ X ├»
       ┌───┐┌───┐ ░                └───┘               └───┘└───┘┌─┴─┐ ░ └───┘»
    o: ┤ X ├┤ H ├─░──────────────────────────────────────────────┤ X ├─░──────»
       └───┘└───┘ ░                                              └───┘ ░      »
c27: 4/═══════════════════════════════════════════════════════════════════════»
                                                                              »
«                                                ░ ┌───┐                ░  ░ »
«  t_0: ──────────────────────■──────────────────░─┤ X ├────────────────░──░─»
«                             │                  ░ └─┬─┘┌───┐           ░  ░ »
«  t_1: ──────────────────────┼────■─────────────░───┼──┤ X ├───────────░──░─»
«                             │    │             ░   │  └─┬─┘┌───┐      ░  ░ »
«  t_2: ──────────────────────┼────┼────■────────░───┼────┼──┤ X ├──────░──░─»
«                             │    │    │        ░   │    │  └─┬─┘┌───┐ ░  ░ »
«  t_3: ──────────────────────┼────┼────┼────■───░───┼────┼────┼──┤ X ├─░──░─»
«                             │    │    │    │   ░   │    │    │  └─┬─┘ ░  ░ »
«  k_0: ──────────────────────┼────┼────┼────┼───░───■────┼────┼────┼───░──░─»
«                             │    │    │    │   ░        │    │    │   ░  ░ »
«  k_1: ──────────────────────┼────┼────┼────┼───░────────■────┼────┼───░──░─»
«                             │    │    │    │   ░             │    │   ░  ░ »
«  k_2: ──────────────────────┼

In [73]:
n = 4
r_text = QuantumRegister(n, 't')
r_key = QuantumRegister(n, 'k')
r_output = QuantumRegister(1, name = 'o')
r_cipher = QuantumRegister(n, 'c')
r_ancilla = QuantumRegister(n, 'a')

r_class = ClassicalRegister(n)

qc = QuantumCircuit(r_text, r_key, r_cipher, r_ancilla, r_output, r_class)
#loading plain text
plain_text = '0101'
load_input(qc, r_text, plain_text, n)

#Preparing key in a superposition state
load_key(qc, r_key, n)

#loading known cipher text
cipher_text = '1001'
load_input(qc, r_cipher, cipher_text, n)

#Preparing output qubit
qc.x(r_output)
qc.h(r_output)
qc.barrier()

#First Iteration
my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
qc.barrier()
qc.append(diffuser(n), r_key)
qc.barrier()

#Second Iteration
my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
qc.barrier()
qc.append(diffuser(n), r_key)
qc.barrier()

qc.measure(r_key, r_class)
#qc.draw()


In [74]:
#sim = AerSimulator(method = 'matrix_product_state')
from qiskit.providers.basic_provider import BasicSimulator
sim = BasicSimulator()
# sim = Aer.get_backend('qasm_simulator')
shots = 1024
#Run and get the counts, using the matrix_product_state method
tcirc = transpile(qc, sim)
result = sim.run(tcirc, shots=1024).result()
print (result.get_counts())
counts = result.get_counts() #counts = result.get_counts(0)
b, c = max(counts.items(), key=lambda x: x[1])
print("Key is :", b[::-1], "with probability =", c/shots)

{'0011': 953, '0000': 2, '0010': 4, '1001': 6, '1100': 7, '0001': 2, '0110': 5, '1101': 4, '1111': 6, '0101': 3, '1011': 6, '1110': 6, '1000': 6, '0111': 6, '1010': 6, '0100': 2}
Key is : 1100 with probability = 0.9306640625


In [75]:
print(result.get_counts(),end="\n")

{'0011': 953, '0000': 2, '0010': 4, '1001': 6, '1100': 7, '0001': 2, '0110': 5, '1101': 4, '1111': 6, '0101': 3, '1011': 6, '1110': 6, '1000': 6, '0111': 6, '1010': 6, '0100': 2}


In [76]:
print("Key is :", b[::-1], "with probability =", c/shots)

Key is : 1100 with probability = 0.9306640625
